In [1]:
from pyteomics import mzml

import tqdm
import pandas as pd

import copy

import glob

import json

import sys

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/akshai/anaconda3/envs/gnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# Read the clustering results
input_file_name = "analysis_results/coverage0.9_correlation0.75.csv"
clustered_data_file = pd.read_csv(input_file_name)
clustered_data_file

,Unnamed: 0,Met_ID,rt_cf,mz_cf,intensity_cf,label,real_name
0,101,102,470.159358,462.963334,261810.40,1137,PFNA
1,172,173,494.488819,512.960155,113655.00,1137,PFDA
2,526,527,470.547127,498.930985,2691497.00,1137,PFOS
3,1068,1069,409.846603,398.936966,1753351.00,211,PFHxS
4,1672,1673,461.647058,498.930997,773771.20,1139,NaN
5,1771,1772,441.325040,368.976961,155059.80,1138,NaN
6,1980,1981,441.239461,412.966987,482334.60,1138,PFOA
7,3206,3207,443.220796,448.934703,91080.49,1139,L-PFHpS
8,4169,4170,515.553913,562.958516,106839.20,1137,PFUdA
9,4423,4424,469.994651,418.974094,55249.51,1137,NaN


In [3]:
# Calculate ppm error between a two masses
def calculate_ppm_error_between_two_masses(exact_mass,observed_mass):
    mass_difference = observed_mass-exact_mass
    ppm_error = (mass_difference * 1000000) / exact_mass
    return (ppm_error)

def find_mass_window(input_mass,mass_windows_dict):
    mass_windows = []
    for key in mass_windows_dict:
        if float(input_mass) >= mass_windows_dict[key]["start"] and float(input_mass) <= mass_windows_dict[key]["end"]:
            mass_windows.append(key)
    return mass_windows

def seconds_to_minutes(input_time):
    return (input_time/60)

def minutes_to_seconds(input_time):
    return (input_time*60)

In [4]:
# Reading MS2 files
ms2_files = [files for files in glob.glob("../MS2/Sample*.mzML") if "FS" not in files]

In [5]:
# Mass windows used and W1, W2 etc are present in the filename
# MS2 is run in windows of masses 
# For each window of mass there are different energy settings
mass_windows_dict = {"W1": {"start":212.5,"end":300},
 "W2": {"start":290,"end":390},
 "W3": {"start":380,"end":480},
 "W4": {"start":470,"end":570},
 "W5": {"start":560,"end":660},
 "W5": {"start":650,"end":750}}

In [6]:
# Create dict containing the key as mass window and values are mass,rt and met id. 
# This is done to make the searching easier by minimizing the opening of files
# The fetched data ms2 etc in the further step are stored in the same dict

found_features = pd.read_csv("found_features.csv")
mass_window_feature_dict = {window:[] for window in mass_windows_dict}
for i,_ in enumerate(tqdm.tqdm(clustered_data_file["mz_cf"].to_list(),leave=True,position=-1)):
    pmz  = clustered_data_file["mz_cf"].to_list()[i]
    rt_input = clustered_data_file["rt_cf"].to_list()[i]
    met_id = clustered_data_file["Met_ID"].to_list()[i]
    mass_windows = find_mass_window(pmz,mass_windows_dict)
    for mass_window in mass_windows:
        cmpd_name = found_features[found_features["met_id"] == met_id]["cmpd_name"].to_list()
        if len(cmpd_name) == 0:
            cmpd_name = "None"
        else:
            cmpd_name = cmpd_name[0]
        mass_window_feature_dict[mass_window].append({"MZ":pmz,"RT":rt_input,"met_id":met_id,"cmpd_name":cmpd_name})


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 3846.93it/s]


In [7]:
ppm_error = 5 # PPM error between precursor mass and input mass
rt_window = 20 # input in seconds, rt window +/- n seconds are considered to fetch ms2 within the region
def fetch_ms2_from_mzml(mz_rt_dict_input,mzml_file_read,file_id):
    mz_rt_dict = mz_rt_dict_input
    
    file_id = str(file_id)
    pmz  = mz_rt_dict["MZ"]
    rt_feature = mz_rt_dict["RT"]
    for i,entry in enumerate(mzml_file_read):
        if entry["ms level"] == 1:
            rt_m1 = minutes_to_seconds(float(entry["scanList"]["scan"][0]['scan start time'])) # Fetch MS1 RT
        if entry["ms level"] == 2:
            selected_mz = float(entry["precursorList"]["precursor"][0]["selectedIonList"]["selectedIon"][0]["selected ion m/z"]) # Get MS2 mass
            if abs(calculate_ppm_error_between_two_masses(pmz,selected_mz)) < ppm_error: # Check if masses are close by with ppm error
                if 0 <= rt_m1 - rt_feature > -rt_window:  # Check if the rt difference is less than zero
                    ms1_index_match = entry["index"]
                    if "rt_start_" + file_id not in mz_rt_dict: # If it is first rt, the mention as rt_start
                        mz_rt_dict["rt_start_index_" + file_id] =  ms1_index_match
                        mz_rt_dict["all_ms2_index_" + file_id] = [] 
                        mz_rt_dict["ms2_index_pmz_rt_" + file_id] = [] 
                        
                    mz_rt_dict["all_ms2_index_" + file_id].append(ms1_index_match) # Add all the rt index for ms2 for the fileid here, there are multiple files with same run (multiple energies) 
                                                                                   #, so need to keep the data separate, hence naming with fileid
                    mz_rt_dict["ms2_index_pmz_rt_" + file_id].append([selected_mz,rt_m1]) # for the index, write the pmz, rt in list form
                    
                if 0 >= rt_m1 - rt_feature > rt_window: # If the rt is greater than zero and last rt in the given window, mention as rt_end
                    ms1_index_match = entry["index"]
                    dicts["rt_end_index_" + file_id] =  ms1_index_match
                    mz_rt_dict["all_ms2_index_" + file_id].append(ms1_index_match)
                    mz_rt_dict["ms2_index_pmz_rt_" + file_id].append([selected_mz,rt_m1])
    
    if "all_ms2_index_" + file_id in mz_rt_dict: # For all the ms2 index fetched, get the m/z and intensity list
        for i,entry in enumerate(mzml_file_read):
            if entry["index"] in mz_rt_dict["all_ms2_index_" + file_id] and entry["ms level"] == 2:
                if "MS2_" + file_id not in mz_rt_dict:
                    mz_rt_dict["MS2_" + file_id] = []
                pmz_intensity = entry['precursorList']["precursor"][0]["selectedIonList"]["selectedIon"][0]["peak intensity"]
                mz_rt_dict["MS2_" + file_id].append({"index":entry["index"],"precursor_intensity":pmz_intensity,"MZ":list(entry["m/z array"]),"INTENSITY":list(entry["intensity array"])}) # Append the information to the dict
    return (mz_rt_dict)

In [8]:
# Fetching the MS2 from the mzML files
loop = tqdm.tqdm(mass_window_feature_dict)
for mass_window in loop: # Get the mass window to run
    mzml_files = []
    for files in ms2_files: # Get the files for the mass window
        if str(mass_window) in files:
            mzml_files.append(files)
    
    for i,file in enumerate(mzml_files): # For each file, run the feature found above (give the dict of mz and rt and will get updated with ms2 if found in the file)
        mzml_file_read = [entry for entry in mzml.read(file)]
        file_id = file.split("/")[-1].split(".")[0][len("Sample_"):]
        for j,dicts in enumerate(mass_window_feature_dict[mass_window]):
            fetch_ms2_from_mzml(dicts,mzml_file_read,file_id)
            loop.set_description(file + "\t" +str(i+1) + "/" + str(len(mzml_files)) + " "+  str(j))
            

../MS2/Sample_W4_NCE10.mzML	9/9 3: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [01:01<00:00, 12.24s/it]


In [9]:
# Save the output into a json file
with open(input_file_name[:-4] + "_ms2.json", 'w', encoding='utf-8') as f:
    json.dump(mass_window_feature_dict, f, ensure_ascii=False, indent=4)

In [10]:
# segregate the above data with met_id (met_id is obtained from feature linker, it is the id for the feature)
# met_id as key and the respective ms2 dict with metadata as value. This is used to produce the mgf file
met_id_data_separation = {}
for windows in mass_window_feature_dict:
    for dicts in mass_window_feature_dict[windows]:
        met_id = dicts["met_id"]
        PEPMASS = dicts["MZ"]
        cmpd_name = dicts["cmpd_name"]
        if met_id not in met_id_data_separation:
            met_id_data_separation[met_id] = []
        
        key = False
        for keys in dicts:
            if "MS2" in keys:
                key = keys
        if key:
            for ms2_dicts in dicts[key]:
                mz_array = ms2_dicts["MZ"]
                intensity_array = ms2_dicts["INTENSITY"]
                met_id_data_separation[met_id].append({"cmpd_name":cmpd_name,"PEPMASS":PEPMASS,"met_id":met_id,"MZ":mz_array,"INTENSITY":intensity_array,"ADDUCT":"[M-H]-","FILE_ID":key[len("MS2_W1_"):],"IONMODE":"Negative"})
            

In [12]:
# Put all the spectrum together
all_data = []
for met_id in met_id_data_separation:
    all_data.extend(met_id_data_separation[met_id])

In [13]:
# Create the mgf file
# Input data of list of dicts
# All the key and value are entered into the mgf file
# MZ and INTENSITY key corresponds to the spectrum
def write_mgf_from_list_of_dict(input_data,filename):
    out_mgf_file = open(filename,"w")
    for entry in input_data:
        write_mgf_from_dict(entry,out_mgf_file)
    out_mgf_file.close()
    
# For a dict containing key:value pairs
# All the key and value are entered into the mgf file
# MZ and INTENSITY key corresponds to the spectrum
# File object has to be given to save the data to a file
def write_mgf_from_dict(input_dict,file_object):
    mgf_file = "BEGIN IONS \n"
    for entry in input_dict:
        if entry != "MZ" and entry != "INTENSITY":
            mgf_file += entry + "=" + str(input_dict[entry]) + "\n"
    
    mz_list = (input_dict["MZ"])
    intensity_list = (input_dict["INTENSITY"])
    
    for i,mz in enumerate(mz_list):
        mgf_file += str(mz) + " " + str(intensity_list[i]) + "\n"
        
    mgf_file += "END IONS\n\n"
    file_object.write(mgf_file)
    
write_mgf_from_list_of_dict(all_data,input_file_name[:-4] + "_ms2.mgf")

In [15]:
# Run one sirius job given mgf file and config file
# Output would be produced in the same folder as the mgf file
def single_sirius_run(mgf_file,config_sirius):
    path = os.path.dirname(os.path.abspath(mgf_file)) + "/"
    output_filename = path + "".join(mgf_file.split("/")[-1].split(".")[:-1]) + ".sirius"
    sirius_command = "sirius --input " \
                        + mgf_file + " --output " + output_filename + \
                        "   " + config_sirius + " > " + path + "sirius.log 2>&1"

    os.system(sirius_command)
    
sirius_config = "config --IsotopeSettings.filter true --FormulaSearchDB ALL_BUT_INSILICO,ALL,BIO,METACYC,CHEBI,COCONUT,ECOCYCMINE,GNPS,HMDB,HSDB,KEGG,KEGGMINE,KNAPSACK,MACONDA,MESH,NORMAN,UNDP,PLANTCYC,PUBCHEM,PUBMED,YMDB,YMDBMINE,ZINCBIO,ADDITIONAL --Timeout.secondsPerTree 0 --FormulaSettings.enforced HCNOFPCl --Timeout.secondsPerInstance 0 --AdductSettings.detectable [[M-H2O-H]-,[M-H]-,[M+Cl]-,[M+Br]-] --UseHeuristic.mzToUseHeuristicOnly 650 --AlgorithmProfile orbitrap --IsotopeMs2Settings IGNORE --MS2MassDeviation.allowedMassDeviation 5.0ppm --NumberOfCandidatesPerIon 1 --UseHeuristic.mzToUseHeuristic 300 --FormulaSettings.detectable B,Cl,Br,Se,S --NumberOfCandidates 10 --StructureSearchDB ALL_BUT_INSILICO,ALL,BIO,METACYC,CHEBI,COCONUT,ECOCYCMINE,GNPS,HMDB,HSDB,KEGG,KEGGMINE,KNAPSACK,MACONDA,MESH,NORMAN,UNDP,PLANTCYC,PUBCHEM,PUBMED,YMDB,YMDBMINE,ZINCBIO,ADDITIONAL --AdductSettings.fallback [[M-H+Na-H]-,[M-H+K-H]-,[M-H2O-H]-,[M+C2H4O2-H]-,[M+C2H3N-H]-,[M-H]-,[M]-,[M+C2HF3O2-H]-,[M+CH2O2-H]-,[M+Br]-,[M+Cl]-] --FormulaResultThreshold true --RecomputeResults false formula structure"

single_sirius_run(input_file_name[:-4] + "_ms2.mgf",sirius_config)